In [ ]:
# --- repo bootstrap ---------------------------------------------------------
from pathlib import Path
from dotenv import load_dotenv
import os, sys

def repo_root(start: Path) -> Path:
    cur = start.resolve()
    while cur != cur.parent:
        if (cur / ".env").exists() or (cur / ".git").exists():
            return cur
        cur = cur.parent
    raise RuntimeError("repo root not found")

ROOT = repo_root(Path.cwd())
load_dotenv(ROOT / ".env")             # loads secrets
sys.path.append(str(ROOT / "src"))     # optional helpers

DATA_DIR = ROOT / "data"
OUT_DIR  = ROOT / "outputs"
FIG_DIR  = OUT_DIR / "figs"; FIG_DIR.mkdir(exist_ok=True)

print("Repo root:", ROOT)

In [ ]:
# """
# Strategy: Run 4 notebooks in parallel, each processing ALL 174k messages
# with a different model. Uses batching to maximize throughput.
# """

# import pandas as pd
# import numpy as np
# from openai import OpenAI
# import asyncio
# import aiohttp
# from pathlib import Path
# import json
# import time
# from datetime import datetime
# from tqdm.notebook import tqdm
# import os
# from dotenv import load_dotenv
# import nest_asyncio
# nest_asyncio.apply()

# load_dotenv()

# # Configuration
# ROOT = Path.cwd().resolve().parents[0] if Path.cwd().name != 'ukraine-final-project' else Path.cwd()
# TELEGRAM_CSV = ROOT / "outputs" / "telegram_full_20250605_213258.csv"
# OUT_DIR = ROOT / "outputs" / "telegram_scoring_parallel"
# OUT_DIR.mkdir(exist_ok=True)

# # SELECT YOUR MODEL FOR THIS NOTEBOOK
# # Change this for each notebook you run in parallel
# MODEL_NAME = "chatgpt-4o-latest"  # Options: "gpt-4o-mini", "o3-mini", "gpt-4.1-mini", "gpt-4.1-nano"

# # Model configurations with batching
# MODEL_CONFIGS = {
#     "gpt-4o-mini": {
#         "rpm": 10000,
#         "tpm": 200000,
#         "batch_size": 5,  # Messages per request
#         "concurrent": 8000,  # Concurrent requests
#         "max_retries": 3
#     },
#     "o3-mini": {
#         "rpm": 500,
#         "tpm": 200000,
#         "batch_size": 25,
#         "concurrent": 20,
#         "max_retries": 3
#     },
#     "gpt-4.1-mini": {
#         "rpm": 500,
#         "tpm": 200000,
#         "batch_size": 25,
#         "concurrent": 20,
#         "max_retries": 3
#     },
#     "gpt-4.1-nano": {
#         "rpm": 500,
#         "tpm": 400000,  # Higher token limit
#         "batch_size": 30,  # Can handle more
#         "concurrent": 20,
#         "max_retries": 3
#     }, 
#     "chatgpt-4o-latest": {
#         "rpm": 200,
#         "tpm": 500000,  # 2.5x more tokens!
#         "batch_size": 10,  # Can batch more
#         "concurrent": 150,  # Near RPM limit
#         "max_retries": 3
#     }
# }

# # Get config for selected model
# CONFIG = MODEL_CONFIGS[MODEL_NAME]

# print(f"🚀 Model Comparison Setup: {MODEL_NAME}")
# print(f"📊 Will process ALL messages with this model")
# print(f"⚡ Batch size: {CONFIG['batch_size']} messages per request")
# print(f"🔄 Concurrent requests: {CONFIG['concurrent']}")

In [ ]:
# # Cell 2: Load Data
# print("\n📊 Loading Telegram data...")
# df = pd.read_csv(TELEGRAM_CSV)
# df = df[df['message_text'].notna()].copy()
# total_messages = len(df)
# print(f"✅ Loaded {total_messages:,} messages")

# # Calculate batches
# total_batches = (total_messages + CONFIG['batch_size'] - 1) // CONFIG['batch_size']
# print(f"📦 Total batches: {total_batches:,}")
# print(f"⏱️  Estimated time: {total_batches / CONFIG['rpm']:.1f} minutes (if rate limited)")

In [ ]:
# BATCH_PROMPT = """War message scorer. Output only: M#:E,B,P,C|...
# E(0-10): 0=humanitarian 3=combat 5=major-weapons 7=nationwide-strikes 9=nuclear-threats
# B(-1/0/1): -1=neutral 0=blames-Ukraine/NATO 1=blames-Russia
# P(0-3): 0=factual 1=spin 2=propaganda 3=extreme-disinfo
# C(0/1): 0=no-action 1=calls-to-action
# Messages:"""

# def create_message_batches(df, batch_size):
#     """Create batches of messages for processing"""
#     batches = []
    
#     for i in range(0, len(df), batch_size):
#         batch_df = df.iloc[i:i+batch_size]
#         batch_messages = []
        
#         for idx, row in batch_df.iterrows():
#             text = str(row['message_text'])[:200].replace('\n', ' ')
#             msg = f"M{idx}:[{row['channel_username']}] {text}"
#             batch_messages.append((idx, msg))
        
#         batches.append(batch_messages)
    
#     return batches

In [ ]:
class RateLimiter:
    def __init__(self, rpm):
        # NO LIMIT - let OpenAI handle rate limiting
        pass
    
    async def acquire(self):
        # Do nothing - just return immediately
        pass

async def process_batch_async(session, batch_messages, model_name, rate_limiter, retry_count=0):
    """Process a batch of messages"""
    await rate_limiter.acquire()
    
    # Create batch prompt
    messages_text = "\n".join([msg for _, msg in batch_messages])
    
    url = "https://api.openai.com/v1/chat/completions"
    headers = {"Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}"}
    
    payload = {
        "model": model_name,
        "messages": [
            {"role": "system", "content": BATCH_PROMPT},
            {"role": "user", "content": messages_text}
        ],
        "temperature": 0,
        "max_tokens": len(batch_messages) * 10  # ~10 tokens per message
    }
    
    try:
        async with session.post(url, json=payload, headers=headers, timeout=30) as response:
            if response.status == 200:
                result = await response.json()
                content = result['choices'][0]['message']['content']
                
                # Parse batch results
                results = {}
                parts = content.split('|')
                
                for part in parts:
                    if ':' in part:
                        try:
                            msg_id, scores = part.split(':')
                            msg_idx = int(msg_id.replace('M', '').strip())
                            score_values = [int(x.strip()) for x in scores.split(',')]
                            
                            if len(score_values) == 4:
                                results[msg_idx] = {
                                    'escalation_score': score_values[0],
                                    'blame_direction': score_values[1],
                                    'propaganda_level': score_values[2],
                                    'has_cta': score_values[3]
                                }
                        except:
                            pass
                
                return results
            
            elif response.status == 429:  # Rate limit
                if retry_count < CONFIG['max_retries']:
                    await asyncio.sleep(2 ** retry_count)
                    return await process_batch_async(session, batch_messages, model_name, rate_limiter, retry_count + 1)
            
    except Exception as e:
        if retry_count < CONFIG['max_retries']:
            await asyncio.sleep(2 ** retry_count)
            return await process_batch_async(session, batch_messages, model_name, rate_limiter, retry_count + 1)
    
    return {}

In [ ]:
async def process_all_messages(df, model_name):
    """Process all messages with real-time progress updates"""
    print(f"\n🚀 Starting processing with {model_name}")
    
    # Create batches
    batches = create_message_batches(df, CONFIG['batch_size'])
    print(f"📦 Created {len(batches):,} batches")
    print(f"⚡ Max concurrent: {CONFIG['concurrent']} (could use up to 8000!)")
    
    # Initialize
    rate_limiter = RateLimiter(CONFIG['rpm'])
    all_results = {}
    failed_batches = 0
    completed_batches = 0
    messages_processed = 0
    
    # Process with massive concurrency
    connector = aiohttp.TCPConnector(limit=0, force_close=True)
    timeout = aiohttp.ClientTimeout(total=30)
    
    async with aiohttp.ClientSession(connector=connector, timeout=timeout) as session:
        # Create ALL tasks at once
        tasks = []
        batch_to_size = {}  # Track batch sizes
        
        for i, batch_messages in enumerate(batches):
            task = asyncio.create_task(
                process_batch_async(session, batch_messages, model_name, rate_limiter)
            )
            tasks.append(task)
            batch_to_size[i] = len(batch_messages)
        
        # Process with REAL-TIME updates
        start_time = time.time()
        
        with tqdm(total=len(batches), desc=f"Batches", position=0) as batch_pbar:
            with tqdm(total=total_messages, desc=f"Messages", position=1) as msg_pbar:
                # Process as they complete (not in order)
                for i, future in enumerate(asyncio.as_completed(tasks)):
                    try:
                        result = await future
                        if isinstance(result, dict) and result:
                            all_results.update(result)
                            messages_processed += len(result)
                            msg_pbar.update(len(result))
                        else:
                            failed_batches += 1
                    except Exception as e:
                        failed_batches += 1
                    
                    completed_batches += 1
                    batch_pbar.update(1)
                    
                    # Update stats every 100 batches
                    if completed_batches % 100 == 0:
                        elapsed = time.time() - start_time
                        rate = completed_batches / elapsed
                        msg_rate = messages_processed / elapsed
                        
                        batch_pbar.set_postfix({
                            'rate': f'{rate:.1f} batch/s',
                            'failed': failed_batches
                        })
                        msg_pbar.set_postfix({
                            'rate': f'{msg_rate:.0f} msg/s',
                            'success': f'{messages_processed/((completed_batches)*CONFIG["batch_size"])*100:.1f}%'
                        })
    
    print(f"\n✅ Completed! Processed {len(all_results):,} messages")
    print(f"❌ Failed batches: {failed_batches:,} ({failed_batches/len(batches)*100:.1f}%)")
    
    return all_results

In [ ]:
# Test your actual rate limit
import openai
client = OpenAI()
try:
    # Make a test request
    response = client.chat.completions.with_raw_response.create(
        model="chatgpt-4o-latest",
        messages=[{"role": "user", "content": "test"}],
        max_tokens=1
    )
    # Check rate limit headers
    print("Rate limit headers:")
    print(f"RPM Limit: {response.headers.get('x-ratelimit-limit-requests')}")
    print(f"TPM Limit: {response.headers.get('x-ratelimit-limit-tokens')}")
    print(f"Remaining: {response.headers.get('x-ratelimit-remaining-requests')}")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Run this to see the actual error
async def debug_test():
    url = "https://api.openai.com/v1/chat/completions"
    headers = {"Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}"}
    
    async def make_request(session):
        payload = {
            "model": "chatgpt-4o-latest",
            "messages": [{"role": "user", "content": "test"}],
            "max_tokens": 1
        }
        try:
            async with session.post(url, json=payload, headers=headers) as resp:
                if resp.status != 200:
                    error_text = await resp.text()
                    return f"Status {resp.status}: {error_text}"
                return "Success"
        except Exception as e:
            return f"Exception: {str(e)}"
    
    async with aiohttp.ClientSession() as session:
        result = await make_request(session)
        print(f"Result: {result}")

# Run it
await debug_test()

In [ ]:
print(f"\n{'='*60}")
print(f"🤖 PROCESSING ALL MESSAGES WITH: {MODEL_NAME}")
print(f"{'='*60}")

start_time = time.time()

# Run async processing
loop = asyncio.get_event_loop()
results = loop.run_until_complete(process_all_messages(df, MODEL_NAME))

# Update dataframe with results
for idx, scores in results.items():
    if idx in df.index:
        for col, value in scores.items():
            df.at[idx, col] = value

# Calculate statistics
elapsed_time = (time.time() - start_time) / 60
n_scored = df['escalation_score'].notna().sum()

print(f"\n{'='*60}")
print(f"✅ PROCESSING COMPLETE FOR {MODEL_NAME}!")
print(f"{'='*60}")
print(f"⏱️  Time elapsed: {elapsed_time:.1f} minutes")
print(f"📊 Messages scored: {n_scored:,} / {total_messages:,}")
print(f"🎯 Success rate: {n_scored/total_messages*100:.1f}%")
print(f"⚡ Processing rate: {n_scored/elapsed_time:.0f} messages/minute")

# Save results
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = OUT_DIR / f"telegram_scored_{MODEL_NAME}_{timestamp}.csv"
df.to_csv(output_file, index=False)
print(f"\n📁 Results saved to: {output_file}")

In [ ]:
# FINE-TUNED MODEL EXPLORATION
"""
This notebook explores your 3 fine-tuned models to understand:
1. What input format they expect
2. How they respond
3. Their consistency and accuracy
4. Speed and cost comparisons
"""

import pandas as pd
import numpy as np
from openai import OpenAI
import time
import json
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Initialize
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    organization="org-d28nmVmBQpF2eNppsJOqaB9l",          # optional if key already tied
    project="proj_SXBV23aZ3XH51x5y1qwF48jV"                # ← critical
)

ROOT = Path.cwd().resolve().parents[0] if Path.cwd().name != 'ukraine-final-project' else Path.cwd()

# Your fine-tuned models
FT_MODELS = {
    "mini": {
        "id": "ft:gpt-4o-mini-2024-07-18:politics-ai-research:ukraine-telegram-mini:BfSq29k1",
        "base": "gpt-4o-mini",
        "train_loss": 0.033,
        "valid_loss": 0.149
    },
    "nano": {
        "id": "ft:gpt-4.1-nano-2025-04-14:politics-ai-research:ukraine-classifier-nano:BfSlvv7Q", 
        "base": "gpt-4.1-nano",
        "train_loss": 0.139,
        "valid_loss": 0.190
    },
    "full": {
        "id": "ft:gpt-4.1-2025-04-14:politics-ai-research:ukraine-classifier:BfStxtYw",
        "base": "gpt-4.1",
        "train_loss": 1.377,  # High train loss - might be undertrained?
        "valid_loss": 0.029   # But low valid loss - interesting!
    }
}

print("🎯 Fine-Tuned Models Loaded:")
for name, info in FT_MODELS.items():
    print(f"\n{name.upper()}:")
    print(f"  Model: {info['id']}")
    print(f"  Base: {info['base']}")
    print(f"  Train/Valid Loss: {info['train_loss']:.3f} / {info['valid_loss']:.3f}")

# %%
# Test 1: Basic Functionality - What format do they expect?
print("\n" + "="*60)
print("TEST 1: INPUT FORMAT DISCOVERY")
print("="*60)

# Test different input formats
test_messages = [
    "Russian forces strike Kyiv infrastructure",  # Simple text
    "[Channel: test] Russian forces strike Kyiv",  # With channel
    "M1: Russian forces strike Kyiv",  # With index
    "Score this: Russian forces strike Kyiv",  # With instruction
]

test_prompts = [
    None,  # No system prompt
    "Score the message",  # Simple instruction
    "Return E,B,P,C scores",  # Specific format
    "You are a war message classifier. Score: E(0-10),B(-1/0/1),P(0-3),C(0/1)"  # Full prompt
]

def test_model_response(model_id, user_msg, system_msg=None):
    """Test how model responds to different inputs"""
    messages = []
    if system_msg:
        messages.append({"role": "system", "content": system_msg})
    messages.append({"role": "user", "content": user_msg})
    
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=messages,
            max_tokens=50,
            temperature=0
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

# Test each model with different formats
for model_name, model_info in FT_MODELS.items():
    print(f"\n\n🔍 Testing {model_name.upper()} Model:")
    print("-" * 50)
    
    for msg in test_messages[:2]:  # Test first 2 message formats
        for prompt in test_prompts[:2]:  # Test first 2 prompt types
            result = test_model_response(model_info['id'], msg, prompt)
            print(f"\nInput: '{msg}'")
            if prompt:
                print(f"System: '{prompt}'")
            print(f"Output: {result}")
            
            # Parse if it looks like scores
            if ',' in result and len(result.split(',')) == 4:
                try:
                    scores = [x.strip() for x in result.split(',')]
                    print(f"Parsed: E={scores[0]}, B={scores[1]}, P={scores[2]}, C={scores[3]}")
                except:
                    pass

# %%
# Test 2: Consistency Check - Same message, multiple runs
print("\n" + "="*60)
print("TEST 2: CONSISTENCY CHECK")
print("="*60)

test_message = "NATO announces new military aid package for Ukraine worth $2 billion"
runs_per_model = 5

consistency_results = {}

for model_name, model_info in FT_MODELS.items():
    print(f"\n🔄 Testing {model_name.upper()} consistency ({runs_per_model} runs):")
    
    results = []
    for i in range(runs_per_model):
        response = test_model_response(model_info['id'], test_message)
        results.append(response)
        print(f"  Run {i+1}: {response}")
    
    # Check if all results are identical
    unique_results = set(results)
    consistency_results[model_name] = {
        'results': results,
        'unique': len(unique_results),
        'consistent': len(unique_results) == 1
    }
    
    print(f"  Consistency: {'✅ PERFECT' if len(unique_results) == 1 else f'⚠️  {len(unique_results)} different outputs'}")

# %%
# Test 3: Speed Comparison
print("\n" + "="*60)
print("TEST 3: SPEED & PERFORMANCE")
print("="*60)

# Load sample messages
df = pd.read_csv(ROOT / "outputs" / "telegram_full_20250605_213258.csv", nrows=100)
test_samples = df[df['message_text'].notna()]['message_text'].tolist()[:10]

speed_results = {}

for model_name, model_info in FT_MODELS.items():
    print(f"\n⚡ Testing {model_name.upper()} speed:")
    
    start_time = time.time()
    responses = []
    
    for msg in test_samples:
        response = test_model_response(model_info['id'], msg[:200])
        responses.append(response)
    
    elapsed = time.time() - start_time
    
    speed_results[model_name] = {
        'total_time': elapsed,
        'avg_time': elapsed / len(test_samples),
        'responses': responses
    }
    
    print(f"  Total time: {elapsed:.2f}s")
    print(f"  Avg per message: {elapsed/len(test_samples)*1000:.0f}ms")
    print(f"  Throughput: {len(test_samples)/elapsed:.1f} messages/second")

# %%
# Test 4: Batch Processing Capability
print("\n" + "="*60)
print("TEST 4: BATCH PROCESSING TEST")
print("="*60)

# Try sending multiple messages at once
batch_formats = [
    # Format 1: Newline separated
    "Message 1: Russia attacks Kyiv\nMessage 2: Peace talks resume\nMessage 3: NATO sends aid",
    
    # Format 2: Numbered
    "1. Russia attacks Kyiv\n2. Peace talks resume\n3. NATO sends aid",
    
    # Format 3: Indexed
    "M0: Russia attacks Kyiv\nM1: Peace talks resume\nM2: NATO sends aid",
]

for model_name, model_info in FT_MODELS.items():
    print(f"\n📦 Testing {model_name.upper()} batch capability:")
    
    for i, batch in enumerate(batch_formats):
        response = test_model_response(model_info['id'], batch)
        print(f"\nFormat {i+1} response: {response}")
        
        # Check if it returned multiple scores
        if '|' in response or '\n' in response or response.count(',') > 4:
            print("  ✅ Appears to handle batches!")
        else:
            print("  ❌ Single response only")

# %%
# Test 5: Edge Cases
print("\n" + "="*60)
print("TEST 5: EDGE CASES")
print("="*60)

edge_cases = [
    "",  # Empty
    "привет",  # Non-English
    "🚀💥",  # Emojis only
    "a" * 500,  # Very long
    "NATO NATO NATO NATO",  # Repetitive
    "2+2=4",  # Non-war content
]

for model_name, model_info in FT_MODELS.items():
    print(f"\n🔧 Testing {model_name.upper()} edge cases:")
    
    for case in edge_cases[:3]:  # Test first 3
        response = test_model_response(model_info['id'], case)
        print(f"  '{case[:20]}...' → {response}")

# %%
# Visualization of Results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Model Loss Comparison
ax = axes[0, 0]
models = list(FT_MODELS.keys())
train_losses = [FT_MODELS[m]['train_loss'] for m in models]
valid_losses = [FT_MODELS[m]['valid_loss'] for m in models]

x = np.arange(len(models))
width = 0.35

ax.bar(x - width/2, train_losses, width, label='Train Loss', alpha=0.8)
ax.bar(x + width/2, valid_losses, width, label='Valid Loss', alpha=0.8)
ax.set_xlabel('Model')
ax.set_ylabel('Loss')
ax.set_title('Training vs Validation Loss')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()

# 2. Speed Comparison
if speed_results:
    ax = axes[0, 1]
    speeds = [speed_results[m]['avg_time'] * 1000 for m in models]
    ax.bar(models, speeds)
    ax.set_xlabel('Model')
    ax.set_ylabel('Avg Response Time (ms)')
    ax.set_title('Response Speed Comparison')

# 3. Consistency Results
if consistency_results:
    ax = axes[1, 0]
    consistency = [consistency_results[m]['unique'] for m in models]
    colors = ['green' if c == 1 else 'orange' for c in consistency]
    ax.bar(models, consistency, color=colors)
    ax.set_xlabel('Model')
    ax.set_ylabel('Number of Unique Outputs')
    ax.set_title('Consistency Test (5 runs, same input)')
    ax.axhline(y=1, color='green', linestyle='--', alpha=0.5)

# 4. Summary Table
ax = axes[1, 1]
ax.axis('tight')
ax.axis('off')

summary_data = []
for model in models:
    summary_data.append([
        model.upper(),
        f"{FT_MODELS[model]['base']}",
        f"{FT_MODELS[model]['train_loss']:.3f}",
        f"{FT_MODELS[model]['valid_loss']:.3f}",
        f"{speed_results.get(model, {}).get('avg_time', 0)*1000:.0f}ms" if speed_results else "N/A"
    ])

table = ax.table(
    cellText=summary_data,
    colLabels=['Model', 'Base', 'Train Loss', 'Valid Loss', 'Avg Speed'],
    cellLoc='center',
    loc='center'
)
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)

plt.suptitle('Fine-Tuned Model Comparison', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# %%
# Final Recommendations
print("\n" + "="*60)
print("📊 ANALYSIS & RECOMMENDATIONS")
print("="*60)

print("\n1. INPUT FORMAT:")
print("   Your models likely expect just the raw message text")
print("   No system prompt needed (it's baked into the fine-tuning)")

print("\n2. OUTPUT FORMAT:")
print("   Models should return: E,B,P,C (4 comma-separated integers)")

print("\n3. BEST MODEL:")
# Analyze which performed best
if consistency_results:
    consistent_models = [m for m in models if consistency_results[m]['consistent']]
    print(f"   Most consistent: {', '.join(consistent_models) if consistent_models else 'None perfectly consistent'}")

if speed_results:
    fastest = min(models, key=lambda m: speed_results[m]['avg_time'])
    print(f"   Fastest: {fastest.upper()} ({speed_results[fastest]['avg_time']*1000:.0f}ms/msg)")

print(f"\n4. CONCERNING OBSERVATIONS:")
if FT_MODELS['full']['train_loss'] > 1.0:
    print("   - 'full' model has high train loss (1.377) - might need more training")
print("   - Test batch processing capability before full-scale run")
print("   - Consider the base model costs (gpt-4.1 is most expensive)")

print("\n5. NEXT STEPS:")
print("   - Run full scoring with best performing model")
print("   - Use batch processing if supported")
print("   - Monitor for consistency issues")

In [ ]:
# Add this to see what project your API key is using
import openai
from openai import OpenAI

client = OpenAI()

# List available models to see what you have access to
try:
    models = client.models.list()
    ft_models = [m for m in models if m.id.startswith('ft:')]
    print(f"Found {len(ft_models)} fine-tuned models:")
    for m in ft_models:
        print(f"  - {m.id}")
except Exception as e:
    print(f"Error listing models: {e}")

# Check your default project
print(f"\nCurrent API key org: {client.organization if hasattr(client, 'organization') else 'Not set'}")

In [ ]:
# Test if it's a project issue
from openai import OpenAI

# Try without any project specification first
client = OpenAI()

# Test the simplest possible call
test_model = "ft:gpt-4o-mini-2024-07-18:politics-ai-research:ukraine-telegram-mini:BfSq29k1"

try:
    response = client.chat.completions.create(
        model=test_model,
        messages=[{"role": "user", "content": "test"}],
        max_tokens=10,
        temperature=0
    )
    print(f"✅ SUCCESS! Response: {response.choices[0].message.content}")
except Exception as e:
    print(f"❌ Error: {e}")
    
    # If that fails, try specifying organization
    try:
        client_with_org = OpenAI(
            organization="org-d28nmVmBQpF2eNppsJOqaB9l"  # From your error message
        )
        response = client_with_org.chat.completions.create(
            model=test_model,
            messages=[{"role": "user", "content": "test"}],
            max_tokens=10
        )
        print(f"✅ SUCCESS with org! Response: {response.choices[0].message.content}")
    except Exception as e2:
        print(f"❌ Still failing: {e2}")

In [ ]:
# List ALL your accessible projects
import requests

headers = {
    "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}",
}

# Get organization details
response = requests.get("https://api.openai.com/v1/organization", headers=headers)
print("Organization info:", response.json())

# Try to get project list (this endpoint might not be public)
response = requests.get("https://api.openai.com/v1/projects", headers=headers)
print("Projects:", response.json() if response.status_code == 200 else "Not accessible")